# 2022-12-09

## Performance Summary
Inital p1: 45.2 ms ± 373 µs 

Initial p2: 196 ms ± 244 µs 

## Initial solution
Woke up att 5:50am (~Midnight UTC-5) and did it on time. 
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
  9   00:46:06   6674      0   03:04:20  11546      0

```

### Puzzle 1

Maintain a dictionary with the positions of the head and the tail and the previous position of the head. Whenever the head is outside of a 3x3 grid centered around the tail, we move the tail to the previous position of the head.

In [8]:
from itertools import product

actions = [x.split(" ") for x in open("data/day-9.txt").read().splitlines()]
pos = dict(head=dict(x=0,y=0), tail=dict(x=0,y=0), head_prev=dict(x=0,y=0))
visited = set(); visited.add((0,0))

vicinity = lambda a, b: any([tuple(pos["head"].values()) == (x+a, b+y) for x,y in product(range(-1, 2), range(-1, 2))])

for dir, steps in actions:
    for _ in range(int(steps)):
        pos["head_prev"] = pos["head"].copy()
        match dir:
            case "L":
                pos["head"]["x"]-=1
            case "U":
                pos["head"]["y"]+=1
            case "R":
                pos["head"]["x"]+=1
            case "D":
                pos["head"]["y"]-=1
        if not vicinity(*pos["tail"].values()):
            pos["tail"] = pos["head_prev"].copy()
            visited.add(tuple(pos["tail"].values()))
len(visited)

6236

### Puzzle 2
We have a dictionary which contains the positions of each knot (including the head). We first perform the action provided by the input. Then we iterativly check for each knot, if the parent knot is in the vicinity of it (within a 3x3 grid). If not, we use their relative position to determine whether we should move diagonally or straight. We update the knots position and then move on to the child knot. If ever, a knot doesn't need to move, then we break since no child will have to move either.

In [9]:
from itertools import product
from numpy import sign

actions = [x.split(" ") for x in open("data/day-9.txt").read().splitlines()]

def solver(n=10):
    pos = list(map(lambda x: x.copy(), [dict(x=0,y=0)]*(n+1)))
    tail_visited = set(); tail_visited.add(tuple(pos[len(pos)-1].values()))
    vicinity = lambda a, b, idx: any([tuple(pos[idx].values()) == (x+a, b+y) for x,y in product(range(-1, 2), range(-1, 2))])

    for dir, steps in actions:
        for _ in range(int(steps)):
            match dir:
                case "L":
                    pos[0]["x"]-=1
                case "U":
                    pos[0]["y"]+=1
                case "R":
                    pos[0]["x"]+=1
                case "D":
                    pos[0]["y"]-=1
            for knot in range(1, len(pos)):
                if not vicinity(*pos[knot].values(), knot-1):
                    relative_parent = [c2-c1 for c1,c2 in zip(pos[knot].values(), pos[knot - 1].values())]
                    pos[knot]["x"] += 1*sign(relative_parent[0])
                    pos[knot]["y"] += 1*sign(relative_parent[1])
                else:
                    break
            tail_visited.add(tuple(pos[len(pos)-1].values()))
    return len(tail_visited)
solver(10)

2242

# Learnings

Today also took quite some time, however, I think that today had the largest time gap between puzzle 1 and puzzle 2. My initial solution for p1 did not work with extention on p2. This is because I assumed that when a knot was not in the vicnity of its parent, it would move to where the parent was. This works in the single not case but not with multiple knots. It took quite some time to figure out that the proper motion of the knots were to move diagonally if the parent was not in the same row or column. I read the example for almost an hour and could not understand why certain moves were done, they felt inconsistent with the explained rules. Anyhow, I think I have a better understanding of the problem now and I think I can solve it in a more elegant way.

- Again, read the problem description carefully.
- Not sure if using dicts here was the right choice. 

# Runtimes

In [6]:
%%timeit
solver(n=1)

45.2 ms ± 373 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
solver(n=10)

196 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
